## Challenge Set 9: Part2
#### *Intermediate level SQL* ####
Subject:      Baseball  
Date:         10/18/2010  
Name:         Steven Bierer  

----

### Create the SSH tunnel and acess the data

In [1]:
# First the SSH tunneling #
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '34.220.227.92'
AWS_USERNAME = 'steve'
SSH_KEY_PATH = '/Users/neuromac/.ssh/id_rsa'    # use "ssh-keygen -p" to set password to nothing

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 50925


In [4]:
# Then create a new SQL engine linked to the BASEBALL database #
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost'
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'steve'
POSTGRES_PASSWORD = 'nathan08'
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)
print(f'Postgres SQL object type {type(cnx)} created.')

Postgres SQL object type <class 'sqlalchemy.engine.base.Engine'> created.


In [5]:
# Take a look at what tables are available in this database #
import pandas as pd

query = ''' SELECT * FROM pg_catalog.pg_tables
            WHERE tableowner = 'steve';'''
pd.read_sql_query(query, cnx)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,allstarfull,steve,None,True,False,False,False
1,public,salaries,steve,None,True,False,False,False
2,public,schools,steve,None,True,False,False,False
3,public,franchises,steve,None,True,False,False,False
4,public,fielding,steve,None,False,False,False,False


### Question 1 -- What was the total spent on salaries by each team, each year?

In [7]:
# Order information alphabetically by team id, with latest year first #
query = ''' SELECT franchid, franchname, yearid, SUM(salary) AS "Total Salaries"
            FROM Salaries
            JOIN Franchises ON Salaries.teamid = Franchises.franchid
            GROUP BY franchid, yearid
            ORDER BY franchid ASC, yearid DESC
            LIMIT 15; '''
pd.read_sql_query(query, cnx)

,franchid,franchname,yearid,Total Salaries
0,ANA,Los Angeles Angels of Anaheim,2004,100534667.0
1,ANA,Los Angeles Angels of Anaheim,2003,79031667.0
2,ANA,Los Angeles Angels of Anaheim,2002,61721667.0
3,ANA,Los Angeles Angels of Anaheim,2001,47535167.0
4,ANA,Los Angeles Angels of Anaheim,2000,51464167.0
5,ANA,Los Angeles Angels of Anaheim,1999,55388166.0
6,ANA,Los Angeles Angels of Anaheim,1998,41281000.0
7,ANA,Los Angeles Angels of Anaheim,1997,31135472.0
8,ARI,Arizona Diamondbacks,2013,90132000.0
9,ARI,Arizona Diamondbacks,2012,73804833.0


### Question 2 -- What is the first and last year played for each player?

First, in the PSGL terminal I entered the following lines. Note, the "cut" command forces only the first few columns to be taken from the .csv file.
```
CREATE TABLE IF NOT EXISTS Fielding (
            playerID varchar(10) NOT NULL,
            yearID int NOT NULL,
            stint int DEFAULT NULL
            );
COPY Fielding FROM PROGRAM 'cut -d "," -f 1,2,3 /home/steve/baseballdata/Fielding.csv' DELIMITER ',' CSV HEADER;
```

In [16]:
# List out all players and their first and last year, in order of their first year played,  #
query = ''' SELECT playerid, MIN(yearid) AS "First Year", MAX(yearid) AS "Last Year"
            FROM Fielding
            GROUP BY playerid
            ORDER BY "First Year", playerid
            LIMIT 5; '''
pd.read_sql_query(query, cnx)

,playerid,First Year,Last Year
0,abercda01,1871,1871
1,addybo01,1871,1877
2,allisar01,1871,1876
3,allisdo01,1871,1883
4,ansonca01,1871,1897


### Question 3 -- Who has played the most all star games?

In [30]:
# List out the players in order of number of A.S. games played. Highest on the list is "AaronHA01" with 25 games. #
# (Whoever that is. The Master.csv file might provide more info like first and last name of the players.)
query = ''' SELECT playerid, COUNT(playerid) AS "Number of Games"
            FROM AllStarFull
            GROUP BY playerid
            ORDER BY "Number of Games" DESC
            LIMIT 5; '''
pd.read_sql_query(query, cnx)

,playerid,Number of Games
0,aaronha01,25
1,mayswi01,24
2,musiast01,24
3,mantlmi01,20
4,willite01,19


### Question 4 -- Which school has generated the most distinct players?

```sql  cnx.execute
```

In [ ]:
# Another new table needs to be created. #
# But this time, instead of executing the SQL commands from the command line, I will run them from Jupyter.